<a href="https://colab.research.google.com/github/BrazilianKing/Food-Sales-Predictions/blob/main/Project_1_part_6_(final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 1 Part 5

Edmar Dos Santos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer, ColumnTransformer 
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import set_config
set_config(display = 'diagram')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/excel downloads/sales_predictions.csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [ ]:
df.nunique()

Item_Identifier              1559
Item_Weight                   415
Item_Fat_Content                5
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

In [ ]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [ ]:
df[df.isna().any(axis=1)].shape

(3873, 12)

In [ ]:
df.duplicated().sum()

0

#Checking value counts for all objetcs then Cordinal encoding each one of them

In [ ]:
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [ ]:
replacement_dictionary1 = {'Low Fat':0, 'LF':0, 'low fat':0, 'Regular':1, 'reg':1}
df['Item_Fat_Content'].replace(replacement_dictionary1, inplace=True)
df['Item_Fat_Content']

0       0
1       1
2       0
3       1
4       0
       ..
8518    0
8519    1
8520    0
8521    1
8522    0
Name: Item_Fat_Content, Length: 8523, dtype: int64

In [ ]:
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [ ]:
replacement_dictionary2 = {'Small':0, 'Medium':1, 'High':2}
df['Outlet_Size'].replace(replacement_dictionary2, inplace=True)
df['Outlet_Size']

0       1.0
1       1.0
2       1.0
3       NaN
4       2.0
       ... 
8518    2.0
8519    NaN
8520    0.0
8521    1.0
8522    0.0
Name: Outlet_Size, Length: 8523, dtype: float64

In [ ]:
df['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [ ]:
replacement_dictionary3 = {'Tier 3':0, 'Tier 2':1, 'Tier 1':2}
df['Outlet_Location_Type'].replace(replacement_dictionary3, inplace=True)
df['Outlet_Location_Type']

0       2
1       0
2       2
3       0
4       0
       ..
8518    0
8519    1
8520    1
8521    0
8522    2
Name: Outlet_Location_Type, Length: 8523, dtype: int64

Identify the features (X) and target (y)

In [ ]:
y = df['Item_Outlet_Sales']

X = df.drop(columns = ['Item_Outlet_Sales', 'Item_Identifier', 'Item_Type', 'Outlet_Identifier'])

# Finding missing data

In [ ]:
print(df.isna().sum().sum(), 'missing values')

3873 missing values


In [ ]:
print(df.isna().sum(), 'missing values')

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64 missing values


#Checking for duplicates

Performing Train/Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#Creatng a preprocessing object to prepare the dataset for Machine Learning

In [ ]:
# Selectors
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [ ]:
# Imputers
freq_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')

# Scaler
scaler = StandardScaler()

# One Hot Encoder
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse = False)

In [ ]:
# Numeric pipeline
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [ ]:
# Categorical pipeline
categorical_pipe = make_pipeline(freq_imputer, ohe)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [ ]:
# Tuples for Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)

In [ ]:
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple, remainder = 'passthrough')
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f080699d880>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f080699d550>)])

In [ ]:
preprocessor.fit(X_train)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f080699d880>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f080699d550>)])

In [ ]:
X_train_proc = preprocessor.transform(X_train)
X_test_proc = preprocessor.transform(X_test)

In [ ]:
X_train_proc[:5]

array([[ 0.81724868, -0.7403206 , -0.71277507,  1.82810922,  1.32784893,
         0.40778536, -1.08494779,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.5563395 ,  1.35076614, -1.29105225,  0.60336888,  1.32784893,
         0.40778536, -1.08494779,  0.        ,  0.        ,  1.        ,
         0.        ],
       [-0.13151196,  1.35076614,  1.81331864,  0.24454056,  0.13618724,
         0.40778536,  1.38477667,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-1.1692189 , -0.7403206 , -1.00493112, -0.95259072,  0.73201809,
        -1.29162953,  0.14991444,  0.        ,  1.        ,  0.        ,
         0.        ],
       [ 1.52881915, -0.7403206 , -0.96548425, -0.33646004,  0.49368575,
         0.        ,  0.14991444,  0.        ,  1.        ,  0.        ,
         0.        ]])

In [ ]:
df = pd.DataFrame(X_train_proc)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.817249,-0.740321,-0.712775,1.828109,1.327849,0.407785,-1.084948,0.0,0.0,1.0,0.0
1,0.556340,1.350766,-1.291052,0.603369,1.327849,0.407785,-1.084948,0.0,0.0,1.0,0.0
2,-0.131512,1.350766,1.813319,0.244541,0.136187,0.407785,1.384777,0.0,1.0,0.0,0.0
3,-1.169219,-0.740321,-1.004931,-0.952591,0.732018,-1.291630,0.149914,0.0,1.0,0.0,0.0
4,1.528819,-0.740321,-0.965484,-0.336460,0.493686,0.000000,0.149914,0.0,1.0,0.0,0.0


#Part 6 Final

###Build a linear regression model to predict sales.

Importing Models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score

#Running your first model

Importing and instantiating the model

In [ ]:
reg = LinearRegression()

Creating a model pipeline

In [ ]:
reg_pipe = make_pipeline(preprocessor,reg)

Training the model on the training data

In [ ]:
reg_pipe.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f080699d880>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f080699d550>)])),
                ('linearregression', LinearRegression())])

Creating model predictions for training and testing data

In [ ]:
train_pred = reg_pipe.predict(X_train)
test_pred = reg_pipe.predict(X_test)

Making a predictions using the testing data

In [ ]:
prediction_df = X_test.copy()
prediction_df['True Median Price'] = y_test
prediction_df['Predicted Median Price'] = test_pred
prediction_df['Error'] = test_pred - y_test
prediction_df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,True Median Price,Predicted Median Price,Error
7503,14.300,0,0.026300,79.4302,1987,2.0,0,Supermarket Type1,1743.0644,1345.677252,-397.387148
2957,7.930,0,0.071136,42.7086,1997,0.0,2,Supermarket Type1,356.8688,746.775963,389.907163
7031,14.500,1,0.041313,42.0454,1999,1.0,2,Supermarket Type1,377.5086,811.523967,434.015367
1084,NaN,1,0.044767,173.7054,1985,1.0,0,Supermarket Type3,5778.4782,4277.334209,-1501.143991
856,10.195,1,0.012456,197.5110,2004,0.0,1,Supermarket Type1,2356.9320,3272.540793,915.608793


## Evaluate the performance of your model based on r^2.

In [ ]:
train_r2 = r2_score(y_train, train_pred)
test_r2 = r2_score(y_test, test_pred)

print(f'Training R2: {train_r2}')
print(f'Testing R2: {test_r2}')

Training R2: 0.559822652647268
Testing R2: 0.5683607931341266


## Evaluate the performance of your model based on rmse.

In [ ]:
train_MSE = mean_squared_error(y_train, train_pred)
test_MSE = mean_squared_error(y_test, test_pred)

print(f'Training RMSE: {train_MSE}')
print(f'Testing RMSE: {test_MSE}')

Training RMSE: 1302685.3616264113
Testing RMSE: 1190882.7682698232


In [46]:
# Using sklearn to find the MSE and including the square root step

train_RMSE = np.sqrt(train_MSE)
test_RMSE = np.sqrt(test_MSE)

print(f'Training RMSE: {train_RMSE}')
print(f'Testing RMSE: {test_RMSE}')

Training RMSE: 1141.3524265652618
Testing RMSE: 1091.2757526261744


# 2. Your second task is to build a regression tree model to predict sales.

## Build a simple regression tree model.

### Importing DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# Making an instance of the model

dec_tree = DecisionTreeRegressor(random_state = 42)

In [ ]:
# Creating pipeline

dec_pipe = make_pipeline(preprocessor,dec_tree)

In [ ]:
# Fit the model using your training data

dec_pipe.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f080699d880>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f080699d550>)])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

Make Predictions

In [ ]:
train_preds = dec_pipe.predict(X_train)
test_preds = dec_pipe.predict(X_test)

Compare the performance of your model based on r^2.

In [ ]:
# Evaluate the model
# r^2

train_score = dec_pipe.score(X_train, y_train)
test_score = dec_pipe.score(X_test, y_test)
print(train_score)
print(test_score)


1.0
0.16254055448476334


Compare the performance of your model based on rmse.

In [ ]:
# Must find MSE inorder to find RMSE

train_MSE = mean_squared_error(y_train, train_pred)
test_MSE = mean_squared_error(y_test, test_pred)

print(f'Training RMSE: {train_MSE}')
print(f'Testing RMSE: {test_MSE}')

Training RMSE: 1302685.3616264113
Testing RMSE: 1190882.7682698232


In [ ]:
# Evaluate RMSE

train_RMSE = np.sqrt(train_MSE)
test_RMSE = np.sqrt(test_MSE)

print(f'Training RMSE: {train_RMSE}')
print(f'Testing RMSE: {test_RMSE}')

Training RMSE: 1141.3524265652618
Testing RMSE: 1091.2757526261744


# Determine which model to implement.

Overall I am going with the model RMSE because RMSE is a common reporting metric for regression models. MSE also has the effect that larger errors become much larger squared errors. An error of 100 would be a squared error of 100^2 =10,000. The benefits of RMSE are that it punishes large errors. EX: MSE and is it is easier to interpret because it is expressed in the same units as the target.